**Name:** Shivam Panchal  
**Company:** Shack Labs  
**Position:** Data Science Internship


# Problem Statement

- Using ML/DL techniques, match similar products from the Flipkart dataset with the Amazon dataset. Once
  similar products are matched, display the retail price from FK and AMZ side by side. Please explore as
  many techniques as possible before choosing the final technique.


## Mounting Drive and Unzipping data


In [3]:
from google.colab import drive

drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd "/content/drive/MyDrive/Work/Shack labs"

/content/drive/MyDrive/Work/Shack labs


In [5]:
pwd


'/content/drive/MyDrive/Work/Shack labs'

In [6]:
!unzip -o "/content/drive/MyDrive/Work/Shack labs/DS - Assignment Part 2 data set.zip" -d "./"

Archive:  /content/drive/MyDrive/Work/Shack labs/DS - Assignment Part 2 data set.zip
  inflating: ./amz_com-ecommerce_sample.csv  
  inflating: ./flipkart_com-ecommerce_sample.csv  


## Method

1. Manually cleaning, stemming and embedding of data and finding similar product based on cosine similarity.
2. Using HuggingFace for Sentence Similarity model and compare similar product name based on cosine similarity.
3. Both dataset has common primary key on which we can merge both the dataset.


### Importing Dataset


In [7]:
import pandas as pd

amz_data = pd.read_csv("./amz_com-ecommerce_sample.csv", encoding="latin-1")
flipkart_data = pd.read_csv("./flipkart_com-ecommerce_sample.csv", encoding="latin-1")


In [8]:
amz_data.head()


,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,982,438,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32143,29121,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,991,551,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,694,325,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,208,258,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [9]:
flipkart_data.head()


,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [10]:
req_col = ["uniq_id", "product_name", "retail_price", "discounted_price"]
amz_df = amz_data[req_col]
flipkart_df = flipkart_data[req_col]


### Method 1: Classic NLP

- Manually cleaning, tokenization, stemming/lemmantizing and embedding.
- matching dataset based on cosine similarity between product name.


#### Importing Libraries


In [11]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer


In [12]:
amz_df1 = amz_df.drop("uniq_id", axis=1)
flipkart_df1 = flipkart_df.drop("uniq_id", axis=1)


In [13]:
nltk.download("stopwords")
stop_words = stopwords.words("english")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#### Preprocessing Data


In [14]:
def prepare_corpus(documents: pd.Series) -> pd.Series:
    """Prepare corpus by preprocessing text

    Args:
        documents (pd.Series): Series of documents

    Returns:
        pd.Series: preprocessed documents
    """
    stemmer = PorterStemmer()
    corpus = (
        documents.str.replace(r"[^a-zA-Z0-9]", " ", regex=True)
        .str.lower()
        .apply(lambda sentence: " ".join([stemmer.stem(word) for word in sentence.split() if word not in stop_words]))
    )
    return corpus


amz_df1["corpus"] = prepare_corpus(amz_df1["product_name"])
flipkart_df1["corpus"] = prepare_corpus(flipkart_df1["product_name"])


#### Embedding with Bag of Words (CountVectorizer)


In [15]:
cv = CountVectorizer()
cv.fit(pd.concat([amz_df1["corpus"], flipkart_df1["corpus"]]))


CountVectorizer()

#### Cosine Similarity between embeddings


In [16]:
cosine_mat = cosine_similarity(cv.transform(amz_df1["corpus"]), cv.transform(flipkart_df1["corpus"]))


#### Concatenating similar products


In [17]:
res_flipkart = []
res_amz = []
for i in range(len(cosine_mat)):
    res_flipkart.append(flipkart_df1.iloc[cosine_mat[i].argmax()])
    res_amz.append(amz_df1.iloc[i])


In [18]:
def resultant_df(data_lists: list, company_names: str) -> pd.DataFrame:
    """Return resultant `DataFrame` which concatenate dataframe and rename `product_name` according to `company_names`

    Args:
        data_lists (list): List of DataFrame
        company_names (str): List of Company Name to rename `product_name`

    Returns:
        pd.DataFrame: resultant `DataFrame`
    """
    resultant_list = []
    for data_list, company in zip(data_lists, company_names):
        resultant_list.append(
            pd.DataFrame(data_list)
            .drop("corpus", axis=1)
            .rename(columns={"product_name": f"{company}_product"})
            .reset_index(drop=True)
        )
    return pd.concat(resultant_list, axis=1)


resultant_df([res_flipkart, res_amz], ["Flipkart", "Amazon"])


,Flipkart_product,retail_price,discounted_price,Amazon_product,retail_price,discounted_price
0,Alisha Solid Women's Cycling Shorts,999.0,379.0,Alisha Solid Women's Cycling Shorts,982,438
1,FabHomeDecor Fabric Double Sofa Bed,32157.0,22646.0,FabHomeDecor Fabric Double Sofa Bed,32143,29121
2,AW Bellies,999.0,499.0,AW Bellies,991,551
3,Alisha Solid Women's Cycling Shorts,999.0,379.0,Alisha Solid Women's Cycling Shorts,694,325
4,Sicons All Purpose Arnica Dog Shampoo,220.0,210.0,Sicons All Purpose Arnica Dog Shampoo,208,258
...,...,...,...,...,...,...
19995,WallDesign Small Vinyl Sticker,1500.0,730.0,WALLDESIGN SMALL VINYL STICKER,1498,876
19996,Wallmantra Large Vinyl Stickers Sticker,1429.0,1143.0,WALLMANTRA LARGE VINYL STICKERS STICKER,1415,1424
19997,Elite Collection Medium Acrylic Sticker,1299.0,999.0,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1284,1196
19998,Elite Collection Medium Acrylic Sticker,1299.0,999.0,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1492,1364


#### Conclusion

- We got table which is concatenated based on similar product of both websites.
- Concatenation/matching is done with `cosine similarity` between word embeddings of product names.
- As matching is done based on product names, product specification is not considered.
- Hence product is purely matched based on it's name i.e. if product has similar name and different specification it will be still matched due to similar name.


### Method 2: Sentence Similarity Models (Hugging Face)

- Using `all-MiniLM-L6-v2` Model for embedding.
- Using cosine similarity


In [19]:
%pip install sentence_transformers --quiet

     |████████████████████████████████| 85 kB 3.7 MB/s 
     |████████████████████████████████| 5.8 MB 39.1 MB/s 
     |████████████████████████████████| 1.3 MB 68.9 MB/s 
     |████████████████████████████████| 182 kB 75.5 MB/s 
     |████████████████████████████████| 7.6 MB 50.7 MB/s 


In [20]:
!nvidia-smi

Fri Dec  2 11:28:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Importing Libraries


In [21]:
from sentence_transformers import SentenceTransformer, util


#### Dataset


In [52]:
amz_df2 = amz_df.drop("uniq_id", axis=1)
flipkart_df2 = flipkart_df.drop("uniq_id", axis=1)


#### Training model and calc cosine similarity


In [53]:
# Instantiate a model of the SentenceTransformer class
model = SentenceTransformer("all-MiniLM-L6-v2")

# Embedding both the Amazon and Flipkart product names and converting them into tensors which will use GPU
amazon_embeds = model.encode(amz_df2["product_name"], convert_to_tensor=True)
flipkart_embeds = model.encode(flipkart_df2["product_name"], convert_to_tensor=True)

cosine_similarities = util.cos_sim(amazon_embeds, flipkart_embeds)


#### Building Dataframe


In [117]:
similar_prod_indexes = []
for cos_vec in cosine_similarities:
    similar_prod_indexes.append(cos_vec.argmax().cpu().numpy())


In [118]:
flipkart_prod_data = []
amz_prod_data = []

for idx, prod_name in enumerate(amz_data.product_name):
    flipkart_prod_data.append(flipkart_df2.iloc[similar_prod_indexes[idx]])
    amz_prod_data.append(amz_df2.iloc[idx].values)


In [119]:
flipkart_prod_df = pd.DataFrame(flipkart_prod_data)
flipkart_prod_df.columns = ["Flipkart Product", "Retail Price in Flipkart", "Discounted Price in Flipkart"]
amz_prod_df = pd.DataFrame(
    amz_prod_data, columns=["Amazon Product", "Retail Price in Amazon", "Discounted Price in Amazon"]
)


In [120]:
pd.concat([amz_prod_df, flipkart_prod_df.reset_index(drop=True)], axis=1)


,Amazon Product,Retail Price in Amazon,Discounted Price in Amazon,Flipkart Product,Retail Price in Flipkart,Discounted Price in Flipkart
0,Alisha Solid Women's Cycling Shorts,982,438,Alisha Solid Women's Cycling Shorts,999.0,379.0
1,FabHomeDecor Fabric Double Sofa Bed,32143,29121,FabHomeDecor Fabric Double Sofa Bed,32157.0,22646.0
2,AW Bellies,991,551,AW Bellies,999.0,499.0
3,Alisha Solid Women's Cycling Shorts,694,325,Alisha Solid Women's Cycling Shorts,999.0,379.0
4,Sicons All Purpose Arnica Dog Shampoo,208,258,Sicons All Purpose Arnica Dog Shampoo,220.0,210.0
...,...,...,...,...,...,...
19995,WALLDESIGN SMALL VINYL STICKER,1498,876,WallDesign Small Vinyl Sticker,1220.0,600.0
19996,WALLMANTRA LARGE VINYL STICKERS STICKER,1415,1424,Wallmantra Large Vinyl Stickers Sticker,1429.0,1143.0
19997,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1284,1196,Elite Collection Medium Acrylic Sticker,1299.0,999.0
19998,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1492,1364,Elite Collection Medium Acrylic Sticker,1299.0,999.0


#### Conclusion

- Same conclusion as Classic NLP method.
- Embeddings is done internally by the model itself.


### Method 3: Merging DataFrame based on `uniq_id`


### Method 4: If all data is stored in Database, Similar product based on name would be retrieved from database api itself.


## For Inputs

- Input drop down for Company/Website Name
- Input from drop down list consisting of Product Names
- For Text input
  - convert text to vector
  - Find similar product name from selected company based on consine similarity of input and product name.
  - From product name select similar matching product from other company/website data.
- `Tkinter GUI`, `Streamlit`, `Flask` or `Fast API` application/API could be use to develop frontend/client side as they are light weight.
